In [1]:
pip install anthropic

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   --------------------- ------------------ 1.0/2.0 MB 3.1 MB/s eta 0:00:01
   ------------------------------- -------- 1.6/2.0 MB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Setup your own API key
* Login & setup your API key from here (https://console.anthropic.com/settings/keys)
* Paste your API key in the following inverted commas

In [ ]:
import os
# Set the environment variable within the notebook
os.environ["ANTHROPIC_API_KEY"] = "your_anthropic_api_key_here"

***Following code is just a sample, your task is to encapsulate this code in a function with arguments of your choice so that function can be reused***

#### Following link is for reference: https://docs.anthropic.com/en/docs/initial-setup#next-steps

In [3]:
import anthropic

client = anthropic.Anthropic()

message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1000,
    temperature=0,
    system="You are a world-class rapper. Respond only with short raps.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Why is the ocean salty?"
                }
            ]
        }
    ]
)
for i in message.content:
    print(i)
print(message.content)


TextBlock(text="Here's a quick rap about salty seas:\n\nYo, listen up while I drop this marine knowledge\nThe ocean's salty, been that way since the college\nOf time began, minerals washing down\nFrom rivers and rocks all around the town\nRain hits the land, dissolves the salt away\nCarries it to seas where it's gonna stay\nVolcanic vents on the ocean floor\nAdding more minerals, yeah there's more\nEvaporation leaves the salt behind\nThat's why the ocean's salty, now you're gonna find\nIt's been building up for billions of years\nMaking waves salty from coast to coast, ya hear?", type='text')
[TextBlock(text="Here's a quick rap about salty seas:\n\nYo, listen up while I drop this marine knowledge\nThe ocean's salty, been that way since the college\nOf time began, minerals washing down\nFrom rivers and rocks all around the town\nRain hits the land, dissolves the salt away\nCarries it to seas where it's gonna stay\nVolcanic vents on the ocean floor\nAdding more minerals, yeah there's mor

In [2]:
#All the required Imports
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Task 1


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


url = "https://www.opentable.com/r/swizzle-louisville?corrid=d8a3a465-39ff-49a5-a866-9a757eb2509a&avt=eyJ2IjoyLCJtIjowLCJwIjowLCJzIjowLCJuIjowfQ&p=2&sd=2024-12-10T19%3A00%3A00&page=1"
driver.get(url)
time.sleep(5)
data = []
b=1
while True:

    reviews = driver.find_elements(By.CSS_SELECTOR, "ol[aria-label='Reviews List'] > li")
    
    for review in reviews:

        reviewer_name = review.find_element(By.CLASS_NAME, '_1p30XHjz2rI-').text

        rating = review.find_element(By.CLASS_NAME, 'yEKDnyk-7-g-').get_attribute('aria-label')

        review_date = review.find_element(By.CLASS_NAME, 'iLkEeQbexGs-').text

        review_text = review.find_element(By.CLASS_NAME, 'l9bbXUdC9v0-').text

        data.append({
            'Reviewer Name': reviewer_name,
            'Rating': rating,
            'Date': review_date,
            'Review Text': review_text
        })
    next_buttons = driver.find_elements(By.CSS_SELECTOR, f'a[aria-label="Go to page number {b+1}"]')
    if next_buttons:
        b+=1
        next_button = next_buttons[0]  
        if next_button.is_displayed() and next_button.is_enabled():
            next_button.click()
            time.sleep(3)  
        else:
            break
    else:
        break 
    print(b)
    if b==214:
        break
driver.quit()

# Save the scraped data to a CSV file
with open('restaurant_reviews_content.csv', 'w', newline='', encoding='utf-8') as file:
    df = pd.DataFrame(data)
    df.to_csv(file, index=False)
print("Scraping complete. Data saved to 'restaurant_reviews_content.csv'.")


# Task 2


In [10]:
import anthropic
import json

client = anthropic.Anthropic()

df = pd.read_csv('restaurant_reviews_content.csv', lineterminator='\n')
df.columns = df.columns.str.strip()

categorized_reviews = []

for index, row in df.iterrows():
    print(index)
    review_text = row["Review Text"]
    
    prompt = (
        "Analyze and categorize the following review from the customer\n"
        "Strictly categorize the reciew into two categories: 1)Comments about food quality,2)Comments about staff/service. Ensure the extracted comments are accurate\n"
        "Ensure no hallucinations occur and no data irrelevant to the category is added (e.g. nostaff related data should be present in the food information and vice versa).\n"

        "Exclude any personal information (PI) of the reviewers. Separate the feedback clearly into two categories: "
        "'Food Quality' and 'Staff/Service'. Avoid giving any additional information or clarity or note from your side, just work on review and give categorizes reponse.If any one or both of the categories are not present then donot provide that category in response only provide that category that is present."
        "The categories should contain the exact text as present in Original review.Donot complete the sentences on your own, donot correct the grammer.Just use exact same text as in original review"
        f"Following is the Review Text: {review_text}\n\n"
        "Provide the response in JSON format like this:\n"
        "{\n"
        "  'Food Quality': 'comments about food',\n"
        "  'Staff/Service': 'comments about staff'\n"
        "}"
        "strictly follow all the instructions.\n"
    )
    try:
        message = client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=1000,
            temperature=0,
            system="You are an expert assistant specialized in text analysis and categorization that has never been wrong. Provide 100 percent accurate responses as asked in the prompts by the user.",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        }
                    ]
                }
            ]
        )
        
        if message:
            response_text = message.content[0].text.strip()
            analysis = json.loads(response_text)
            categorized_reviews.append({
                "Reviewer Name": row["Reviewer Name"],
                "Rating": row["Rating"],
                "Date": row["Date"],
                "Original Review": review_text,
                "Analysis": analysis
            })
        else:
            print(f"Message not generated for index {index}. Skipping...")
    
    except Exception as e:
        print(f"Error processing review at index {index}: {e}")
        continue

    time.sleep(2)

output_file = 'categorized_reviews.json'
with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(categorized_reviews, file, indent=4)

print(f"Categorized reviews have been saved to '{output_file}'.")

0
1
2
3
4
5
6
Error processing review at index 6: Expecting value: line 1 column 1 (char 0)
7
8
9
10
11
12
13
Error processing review at index 13: Extra data: line 4 column 1 (char 5)
14
15
16
17
18
Error processing review at index 18: Expecting property name enclosed in double quotes: line 2 column 3 (char 4)
19
20
Error processing review at index 20: Error code: 500 - {'type': 'error', 'error': {'type': 'api_error', 'message': 'Internal server error'}}
21
22
Error processing review at index 22: Expecting value: line 1 column 1 (char 0)
23
24
25
26
27
28
29
30
31
Error processing review at index 31: Extra data: line 4 column 1 (char 5)
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Error processing review at index 50: Extra data: line 4 column 1 (char 5)
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
Error processing review at index 89: Expecting value: line 1 column 1 (char 0)
90
91
92
93
94
95
96
97
Erro

# Task 3


In [5]:
from flask import Flask, render_template, request, jsonify
import json
import re
from math import ceil
app = Flask(__name__)

with open('categorized_reviews.json', 'r', encoding='utf-8') as file:
    reviews_data = json.load(file)

def preprocess_reviews(reviews_data):
    for review in reviews_data:

        original_review = re.sub(r'\s+', ' ', review['Original Review'].strip())
        food_quality = re.sub(r'\s+', ' ', review['Analysis'].get('Food Quality', '').strip())
        staff_service = re.sub(r'\s+', ' ', review['Analysis'].get('Staff/Service', '').strip())

        for sentence in re.split(r'[.!?]', food_quality):
            sentence = sentence.strip()
            if sentence and sentence in original_review:
                original_review = original_review.replace(
                    sentence, f'<span class="food">{sentence}</span>'
                )

        for sentence in re.split(r'[.!?]', staff_service):
            sentence = sentence.strip()
            if sentence and sentence in original_review:
                original_review = original_review.replace(
                    sentence, f'<span class="staff">{sentence}</span>'
                )

        if food_quality and staff_service:
            review['Label'] = 'both' 
        elif food_quality:
            review['Label'] = 'food'  
        elif staff_service:
            review['Label'] = 'staff'  
        else:
            review['Label'] = 'none'  

        review['Original Review'] = original_review
    return reviews_data


reviews_data = preprocess_reviews(reviews_data)

@app.route('/', methods=['GET', 'POST'])
def index():
    search_query = request.form.get('search_query', '').strip().lower()
    page = int(request.args.get('page', 1)) 
    reviews_per_page = 10  
    filtered_reviews = []

    if search_query:
        for review in reviews_data:
            label = review['Label']

            if search_query == 'food' and label in ['food', 'both']:
                filtered_reviews.append(review)
            elif search_query in ['staff', 'service','services'] and label in ['staff', 'both']:
                filtered_reviews.append(review)
            elif search_query in review['Original Review'].lower():
                filtered_reviews.append(review)
    else:
        filtered_reviews = reviews_data

    total_reviews = len(filtered_reviews)
    total_pages = ceil(total_reviews / reviews_per_page)
    start = (page - 1) * reviews_per_page
    end = start + reviews_per_page
    paginated_reviews = filtered_reviews[start:end]

    pagination = {
        "current_page": page,
        "total_pages": total_pages,
        "prev_page": page - 1 if page > 1 else None,
        "next_page": page + 1 if page < total_pages else None,
        "pages": list(range(1, total_pages + 1))
    }

    return render_template('index.html', reviews=paginated_reviews, pagination=pagination)

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, app)

 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Dec/2024 15:18:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2024 15:18:44] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2024 15:18:44] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2024 15:18:50] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2024 15:18:50] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2024 15:19:04] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2024 15:19:04] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2024 15:19:23] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2024 15:19:23] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2024 15:19:23] "GET /static/style.css HTTP/1.1" 304 -


# Task 4


In [ ]:
from flask import Flask, render_template, request, jsonify
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from datetime import datetime, timedelta
import json
import re
from math import ceil
import matplotlib.pyplot as plt

app = Flask(__name__)

with open('categorized_reviews.json', 'r', encoding='utf-8') as file:
    reviews_data = json.load(file)

def preprocess_reviews(reviews_data):
    for review in reviews_data:
        original_review = re.sub(r'\s+', ' ', review['Original Review'].strip())
        food_quality = re.sub(r'\s+', ' ', review['Analysis'].get('Food Quality', '').strip())
        staff_service = re.sub(r'\s+', ' ', review['Analysis'].get('Staff/Service', '').strip())

        for sentence in re.split(r'[.!?]', food_quality):
            sentence = sentence.strip()
            if sentence and sentence in original_review:
                original_review = original_review.replace(
                    sentence, f'<span class="food">{sentence}</span>'
                )

        for sentence in re.split(r'[.!?]', staff_service):
            sentence = sentence.strip()
            if sentence and sentence in original_review:
                original_review = original_review.replace(
                    sentence, f'<span class="staff">{sentence}</span>'
                )

        if food_quality and staff_service:
            review['Label'] = 'both' 
        elif food_quality:
            review['Label'] = 'food'  
        elif staff_service:
            review['Label'] = 'staff'  
        else:
            review['Label'] = 'none'  

        review['Original Review'] = original_review
    return reviews_data


reviews_data = preprocess_reviews(reviews_data)

@app.route('/', methods=['GET', 'POST'])
def index():
    search_query = request.form.get('search_query', '').strip().lower()
    page = int(request.args.get('page', 1)) 
    reviews_per_page = 10  
    filtered_reviews = []

    if search_query:
        for review in reviews_data:
            label = review['Label']

            if search_query == 'food' and label in ['food', 'both']:
                filtered_reviews.append(review)
            elif search_query in ['staff', 'service','services'] and label in ['staff', 'both']:
                filtered_reviews.append(review)
            elif search_query in review['Original Review'].lower():
                filtered_reviews.append(review)
    else:
        filtered_reviews = reviews_data

    total_reviews = len(filtered_reviews)
    total_pages = ceil(total_reviews / reviews_per_page)
    start = (page - 1) * reviews_per_page
    end = start + reviews_per_page
    paginated_reviews = filtered_reviews[start:end]

    pagination = {
        "current_page": page,
        "total_pages": total_pages,
        "prev_page": page - 1 if page > 1 else None,
        "next_page": page + 1 if page < total_pages else None,
        "pages": list(range(1, total_pages + 1))
    }

    return render_template('index.html', reviews=paginated_reviews, pagination=pagination)

def parse_date(date_string):
    if "Dined on" in date_string:
        return datetime.strptime(date_string.replace("Dined on ", "").strip(), "%B %d, %Y")
    elif "days ago" in date_string:
        days_ago = int(date_string.split()[1])
        return datetime.now() - timedelta(days=days_ago)
    else:
        return None


@app.route('/competitor-analysis', methods=["GET", "POST"])
def competitor_analysis():
    if request.method == "POST":
        competitor_url = request.form["competitor_url"]
        print(f"Competitor URL: {competitor_url}")

        file_name = scrape_reviews(competitor_url)

        df = pd.read_csv(file_name, lineterminator='\n')
        df.columns = df.columns.str.strip()

        swizzle_data = pd.read_csv('restaurant_reviews_content.csv')
        swizzle_data.columns = swizzle_data.columns.str.strip()

        rating=[]
        for i in swizzle_data['Rating']:
            rating.append(int(i[0]))
        
        swizzle_data['Rating']=rating
        swizzle_data['Date'] = swizzle_data['Date'].apply(parse_date)

        competitor_data = pd.DataFrame()
        rating=[]

        for i in df['Rating']:
            rating.append(int(i[0]))
        if len(rating) < len(swizzle_data):
            rating.extend([None] * (len(swizzle_data) - len(rating)))
        elif len(rating) > len(swizzle_data):
            rating = rating[:len(swizzle_data)]

        competitor_data['Rating']=rating

        dates = list(df['Date'].apply(parse_date))
        if len(dates) < len(swizzle_data):
            dates.extend([None] * (len(swizzle_data) - len(dates)))
        elif len(dates) > len(swizzle_data):
            dates = dates[:len(swizzle_data)]

        competitor_data['Date'] = dates


        swizzle=pd.DataFrame({
            'Rating':swizzle_data['Rating'],
            'Date':swizzle_data['Date']
        })

        swizzle_monthly = swizzle.groupby(swizzle['Date'].dt.to_period('M'))['Rating'].mean()
        competitor_monthly=competitor_data.groupby(competitor_data['Date'].dt.to_period('M'))['Rating'].mean()

        comparison_df = pd.DataFrame({
            'Swizzle': swizzle_monthly,
            'Competitor': competitor_monthly
        })
        comparison_df.fillna(0, inplace=True)

        #Plotting
        plt.figure(figsize=(10, 6))
        plt.plot(comparison_df.index.astype(str), comparison_df['Swizzle'], label="Swizzle Restaurant", marker='o')
        plt.plot(comparison_df.index.astype(str), comparison_df['Competitor'], label="Competitor Restaurant", marker='o')

        plt.xlabel('Month')
        plt.ylabel('Average Rating')
        plt.title('Competitor Analysis: Rating Comparison Over Time')
        plt.xticks(rotation=90)
        plt.legend()
        plt.tight_layout()

        # Save the plot as a static image
        plot_file = 'static/comparison_plot.png'
        plt.savefig(plot_file)

        return render_template("competitor_analysis.html", plot_url=plot_file)

    return render_template("competitor_analysis.html")

def scrape_reviews(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    time.sleep(5)

    data = []
    b=1
    while True:
        try:
            reviews = driver.find_elements(By.CSS_SELECTOR, "ol[aria-label='Reviews List'] > li")
            
            for review in reviews:

                rating = review.find_element(By.CLASS_NAME, 'yEKDnyk-7-g-').get_attribute('aria-label')
                review_date = review.find_element(By.CLASS_NAME, 'iLkEeQbexGs-').text

                data.append({
                    'Rating': rating,
                    'Date': review_date,
                })
            next_buttons = driver.find_elements(By.CSS_SELECTOR, f'a[aria-label="Go to page number {b+1}"]')
            if next_buttons:
                b+=1
                next_button = next_buttons[0]  
                if next_button.is_displayed() and next_button.is_enabled():
                    next_button.click()
                    time.sleep(3)  
                else:
                    break
            else:
                break 
            
            print(b)
        except StaleElementReferenceException:
            print("StaleElementReferenceException: Retrying...")
            continue
        except TimeoutException:
            print("Timeout waiting for reviews or next button.")
            break
        except Exception as e:
            print(f"Unexpected error: {e}")
            break
    driver.quit()

    outputfile='restaurant_2.csv'
    with open(outputfile, 'w', newline='', encoding='utf-8') as file:
        df = pd.DataFrame(data)
        df.to_csv(file, index=False)

    return outputfile



if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, app)


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Dec/2024 11:46:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2024 11:46:18] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [13/Dec/2024 11:46:25] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2024 11:46:25] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [13/Dec/2024 11:46:29] "GET /?page=160 HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2024 11:46:29] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [13/Dec/2024 11:46:51] "GET /?page=188 HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2024 11:46:51] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [13/Dec/2024 11:46:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2024 11:46:57] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [13/Dec/2024 11:47:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2024 11:47:03] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [13/Dec/2024 11:47:07] "GET /?page=148 HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2024 11:47:07] "

Competitor URL: https://www.opentable.com/r/grand-lake-kitchen-dimond-oakland?originId=67f07da5-0bad-4118-a273-7b3850062821&corrid=67f07da5-0bad-4118-a273-7b3850062821&avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ
